In [51]:
import pandas as pd

In [52]:
dataset=pd.read_csv("prep.csv",index_col=None)
dataset=pd.get_dummies(dataset,drop_first=True)

In [53]:
x_indep=dataset.drop('classification_yes',axis=1)
y_dep=dataset['classification_yes']

In [62]:
def split_preprocess(x_indep,y_dep):
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x_indep, y_dep, test_size = 0.25, random_state = 0)
    
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train, x_test, y_train, y_test

def selectKBest(x_indep,y_dep,n):
    from sklearn.feature_selection import SelectKBest,chi2
    skb=SelectKBest(score_func=chi2,k=n)
    skb1=skb.fit(x_indep,y_dep)
    selectk_features=skb1.transform(x_indep)
    return selectk_features

def cm_prediction(classifier,x_test,y_test):
    y_pred=classifier.predict(x_test)
    from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
    ascore=accuracy_score(y_test,y_pred)
    cm=confusion_matrix(y_test,y_pred)
    creport=classification_report(y_test,y_pred)
    return ascore,cm,creport
    

def logistic(x_train,y_train,x_test):       
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(x_train, y_train)
    ascore,cm,creport=cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport

def svm_linear(x_train,y_train,x_test):       
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear',random_state=0)
    classifier.fit(x_train, y_train)
    ascore,cm,creport=cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport

def svm_NL(x_train,y_train,x_test):       
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf',random_state=0)
    classifier.fit(x_train, y_train)
    ascore,cm,creport=cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport

def Navie(x_train,y_train,x_test):       
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(x_train, y_train)
    ascore,cm,creport =cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport          
    
def knn(x_train,y_train,x_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(x_train, y_train)
    ascore,cm,creport =cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport 
    
def Decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(x_train, y_train)
    ascore,cm,creport =cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport  

def Random(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(x_train, y_train)
    ascore,cm,creport =cm_prediction(classifier,x_test,y_test)
    return ascore,cm,creport 
    
def classification_report(acclog,accsvml,accsvmnl,accnb,accknn,accdt,accrf):
    result=pd.DataFrame(index=['chiSquare'],columns=['Logistic','accsvml','accsvmnl','accnb','accknn','accdt','accrf'])
    
    for number,idx in enumerate(result.index):
        result['Logistic'][idx]=acclog[number]
        result['accsvml'][idx]=accsvml[number]
        result['accsvmnl'][idx]=accsvmnl[number]
        result['accnb'][idx]=accnb[number]
        result['accknn'][idx]=accknn[number]
        result['accdt'][idx]=accdt[number]
        result['accrf'][idx]=accrf[number]
    return result

In [63]:
acclog=[]
accsvml=[]
accsvmnl=[]
accnb=[]
accknn=[]
accdt=[]
accrf=[]

kbest=selectKBest(x_indep,y_dep,5)
x_train, x_test, y_train, y_test = split_preprocess(kbest,y_dep)

ascore,cm,creport=logistic(x_train,y_train,x_test)
acclog.append(ascore)

ascore,cm,creport=svm_linear(x_train,y_train,x_test)
accsvml.append(ascore)

ascore,cm,creport=svm_NL(x_train,y_train,x_test)
accsvmnl.append(ascore)

ascore,cm,creport=Navie(x_train,y_train,x_test)
accnb.append(ascore)

ascore,cm,creport=knn(x_train,y_train,x_test)
accknn.append(ascore)

ascore,cm,creport=Decision(x_train,y_train,x_test)
accdt.append(ascore)

ascore,cm,creport=Random(x_train,y_train,x_test)
accrf.append(ascore)

result=classification_report(acclog,accsvml,accsvmnl,accnb,accknn,accdt,accrf)
print(result)

          Logistic accsvml accsvmnl accnb accknn accdt accrf
chiSquare     0.94    0.94     0.95  0.83   0.89  0.96  0.95
